In [6]:
import pandas as pd
import hvplot.pandas
import panel as pn

classes_url = (
    "https://raw.githubusercontent.com/leedrake5/Russia-Ukraine/main/data/classes.csv"
)
classes = pd.read_csv(classes_url)

totals_by_systems_url = (
    "https://raw.githubusercontent.com/scarnecchia/oryx_data/main/totals_by_system.csv"
)
totals_by_systems = pd.read_csv(totals_by_systems_url)

oryx = pd.merge(
    totals_by_systems, classes, how="inner", left_on="system", right_on="system"
)

oryx["date_recorded"] = pd.to_datetime(oryx["date_recorded"])
oryx["class"] = oryx["class"].astype("category")
oryx["country_x"] = oryx["country_x"].astype("category")
oryx["system"] = oryx["system"].astype("category")
oryx["origin"] = oryx["origin"].astype("category")
oryx.rename(columns={"country_y": "Country"}, inplace=True)
ukraine_color = "#b94d4d"
russia_color = "#0057b7"
# oryx.dtypes


In [2]:
classes = oryx.groupby(["class", "Country"]).agg(count=("url", "count"))

overall_plot = classes.hvplot(
    kind="barh",
    stacked=True,
    height=500,
    legend="bottom_right",
    title="Overall Equipment Losses",
    color=[ukraine_color, russia_color],
)
overall_plot


:Bars   [class,Country]   (count)

In [11]:
step_one = oryx.groupby(["date_recorded", "Country"], as_index=False).count()
step_two = pd.pivot_table(
    step_one, values="country_x", index="date_recorded", columns="Country"
)
cumulative_losses_plot = step_two.cumsum().hvplot.line(
    x="date_recorded",
    y=["Ukraine", "Russia"],
    # value_label='Equipment Lost',
    legend="top",
    height=600,
    width=920,
    xlabel="Date Recorded",
    ylabel="Equipment Lost",
    title="Cumulative Losses",
    color=[ukraine_color, russia_color],
)

cumulative_losses_plot


:NdOverlay   [Country]
   :Curve   [date_recorded]   (value)

In [14]:
#step_two.cumsum().to_csv('daily_losses.csv')

# bootstrap = pn.template.BootstrapTemplate(title='Minecraft Summary Stats')
bootstrap = pn.template.BootstrapTemplate(
    title="Minecraft Summary Stats", theme=pn.template.bootstrap.BootstrapDarkTheme
)

md = pn.pane.Markdown(
    """
This dashboard summarizes Minecraft Java Edition 
statistics based on local saved data. This data aincludes saved worlds on a local
machine, and the Minecraft log files. As of Spring 2023, this is a work in progress
and the plots and layouts are in flux.

Visit the [GitHub page](https://github.com/CulleyHarrelson/MinecraftStats) to
create a version based on your local system.

"""
)

bootstrap.sidebar.append(pn.Spacer(height=90))
bootstrap.sidebar.append(md)
bootstrap.main.append(overall_plot)
bootstrap.main.append(cumulative_losses_plot)

bootstrap.servable()



BootstrapTemplate
    [js_area] HTML(None, height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] TemplateActions()
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [6026209872] Spacer(height=90)
    [5980547856] Markdown(str)
    [5980284240] HoloViews(Bars, height=500, sizing_mode='fixed', theme=<bokeh.themes.theme.Theme ..., width=700)
    [6026340240] HoloViews(NdOverlay, height=600, sizing_mode='fixed', theme=<bokeh.themes.theme.Theme ..., width=920)